In [9]:
!pip install keras

In [6]:
%matplotlib inline

import matplotlib.pylab as plt

from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [13]:
df_features

time__variance_larger_than_standard_deviation  time__has_duplicate_max  \
1 2                                            0.0                      0.0   
  3                                            1.0                      0.0   
  4                                            1.0                      0.0   
  5                                            1.0                      0.0   

     time__has_duplicate_min  time__has_duplicate  time__sum_values  \
1 2                      0.0                  0.0               6.0   
  3                      0.0                  0.0              10.0   
  4                      0.0                  0.0              18.0   
  5                      0.0                  0.0              27.0   

     time__abs_energy  time__mean_abs_change  time__mean_change  \
1 2              14.0                   1.00               1.00   
  3              30.0                   1.00               1.00   
  4              94.0                   1.75               1.75   
  5             175.0                   1.60               1.60   

     time__mean_second_derivative_central  time__median  ...  \
1 2                                   0.0           2.0  ...   
  3                                   0.0           2.5  ...   
  4                                   0.5           3.0  ...   
  5                                   0.0           3.5  ...   

     x__permutation_entropy__dimension_6__tau_1  \
1 2                                         NaN   
  3                                         NaN   
  4                                         NaN   
  5                                        -0.0   

     x__permutation_entropy__dimension_7__tau_1  \
1 2                                         NaN   
  3                                         NaN   
  4                                         NaN   
  5                                         NaN   

     x__query_similarity_count__query_None__threshold_0.0  \
1 2                                                NaN      
  3                                                NaN      
  4                                                NaN      
  5                                                NaN      

     x__matrix_profile__feature_"min"__threshold_0.98  \
1 2                                               NaN   
  3                                               NaN   
  4                                               NaN   
  5                                               NaN   

     x__matrix_profile__feature_"max"__threshold_0.98  \
1 2                                               NaN   
  3                                               NaN   
  4                                               NaN   
  5                                               NaN   

     x__matrix_profile__feature_"mean"__threshold_0.98  \
1 2                                                NaN   
  3                                                NaN   
  4                                                NaN   
  5                                                NaN   

     x__matrix_profile__feature_"median"__threshold_0.98  \
1 2                                                NaN     
  3                                                NaN     
  4                                                NaN     
  5                                                NaN     

     x__matrix_profile__feature_"25"__threshold_0.98  \
1 2                                              NaN   
  3                                              NaN   
  4                                              NaN   
  5                                              NaN   

     x__matrix_profile__feature_"75"__threshold_0.98  \
1 2                                              NaN   
  3                                              NaN   
  4                                              NaN   
  5                                              NaN   

     x__mean_n_absolute_max__number_of_maxima_7  
1 

In [12]:
df_rolled

,id,time,x,y,sort
9,"(1, 2)",1,1,5,0
10,"(1, 2)",2,2,6,1
11,"(1, 2)",3,3,7,2
5,"(1, 3)",1,1,5,0
6,"(1, 3)",2,2,6,1
7,"(1, 3)",3,3,7,2
8,"(1, 3)",4,4,8,3
0,"(1, 4)",1,1,5,0
1,"(1, 4)",2,2,6,1
2,"(1, 4)",3,3,7,2


In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error

2022-08-01 10:24:10.891844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 10:24:10.891872: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [34]:
input_df = pd.read_csv("data/input_data_nn.csv", sep=";")
output_df = pd.read_csv("data/output_data_nn.csv", sep=";")

In [35]:
df = pd.concat([input_df, output_df], axis=1)

In [36]:
df['time'] = pd.date_range(start='2021-01-01 00:30:00', end='2021-12-31 23:30:00', freq='H')
df['time'] = df['time'].astype(str)

In [37]:
cols = [
 'pot_reativa_inj_barramento11',
 'pot_reativa_inj_barramento12',
 'pot_reativa_inj_barramento24',
 'pot_ativa_inj_barramento11',
 'pot_ativa_inj_barramento12',
 'pot_ativa_inj_barramento24',    
 'tensao_barramento11',
 'tensao_barramento12',
 'tensao_barramento24',
 'carga_subrede_138kv',
 'pot_eolica_subrede_138kv',
 'pot_solar_subrede_138kv',
 'time',
# 'status_1_gerador_subrede_230kv',
# 'status_2_gerador_subrede_230kv',
# 'status_3_gerador_subrede_230kv',
# 'status_4_gerador_subrede_230kv',
# 'status_5_gerador_subrede_230kv',
# 'status_6_gerador_subrede_230kv',
#  'status_7_gerador_subrede_230kv',
#  'status_8_gerador_subrede_230kv',
#  'status_9_gerador_subrede_230kv',
#  'status_10_gerador_subrede_230kv',
#  'status_11_gerador_subrede_230kv',
#  'status_12_gerador_subrede_230kv',
#  'status_13_gerador_subrede_230kv',
#  'status_14_gerador_subrede_230kv',
#  'status_15_gerador_subrede_230kv',
#  'status_16_gerador_subrede_230kv',
#  'status_17_gerador_subrede_230kv',
#  'status_18_gerador_subrede_230kv',
#  'status_1_linha_subrede_230kv',
#  'status_2_linha_subrede_230kv',
#  'status_3_linha_subrede_230kv',
#  'status_4_linha_subrede_230kv',
#  'status_5_linha_subrede_230kv',
#  'status_6_linha_subrede_230kv',
#  'status_7_linha_subrede_230kv',
#  'status_8_linha_subrede_230kv',
#  'status_9_linha_subrede_230kv',
#  'status_10_linha_subrede_230kv',
#  'status_11_linha_subrede_230kv',
#  'status_12_linha_subrede_230kv',
#  'status_13_linha_subrede_230kv',
#  'status_14_linha_subrede_230kv',
#  'status_15_linha_subrede_230kv',
#  'status_16_linha_subrede_230kv',
#  'status_17_linha_subrede_230kv',
#  'status_18_linha_subrede_230kv',
#  'status_19_linha_subrede_230kv',
#  'status_20_linha_subrede_230kv',
#  'status_21_linha_subrede_230kv'
]

df = df[cols]

In [38]:
x, y = df[cols[1:]], df[cols[0]] # FIRST SHIFT AND THEN EXTRACT

In [16]:
x.values

array([[-11.8390979572755, 32.8175413158813, 26.6537126949103,
        29.7205378591306, 89.041938766098, '2021-01-01 00:30:00'],
       [-16.8537310553602, 26.6241219662525, -3.70390773483942,
        -6.40512710013627, 15.0413713227174, '2021-01-01 01:30:00'],
       [-17.1278073706232, 24.970313993721, 1.80549082650761,
        -0.938762652862687, 18.5662200364983, '2021-01-01 02:30:00'],
       ...,
       [-24.9239743638724, 28.5200396621579, 72.6033573829872,
        55.0292414914167, 122.56899235361, '2021-12-31 21:30:00'],
       [-16.2620717414951, 25.8414845351298, 57.8736080497443,
        31.3132672226518, 84.5905949623942, '2021-12-31 22:30:00'],
       [-16.6367658725756, 25.9937710525532, 56.8337202356547,
        30.1533493313895, 83.7436941053097, '2021-12-31 23:30:00']],
      dtype=object)

In [17]:
extraction_settings = ComprehensiveFCParameters()

X = extract_features(x, column_id='time', column_sort='time',
                     default_fc_parameters=extraction_settings,
                     # we impute = remove all NaN features automatically
                     impute_function=impute)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [04:51<00:00, 14.58s/it]


In [19]:
X_filtered = select_features(X, y)
X_fittered.head()

ValueError: Index of X and y must be identical if provided

In [20]:
X.shape

(8760, 3945)

In [21]:
y.shape

(8760,)

In [27]:
from tsfresh.examples import robot_execution_failures

robot_execution_failures.download_robot_execution_failures()
df, y = robot_execution_failures.load_robot_execution_failures()
df.head()

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0


In [82]:
df.sha

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])

In [30]:
extraction_settings = ComprehensiveFCParameters()

X = extract_features(df, column_id='id', column_sort='time',
                     default_fc_parameters=extraction_settings,
                     # we impute = remove all NaN features automatically
                     impute_function=impute)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.34it/s]


In [25]:
X.shape, y.shape

((15, 5523), (88,))

In [32]:
X_filtered = select_features(X, y)
X_filtered.head()

,F_x__value_count__value_-1,F_x__abs_energy,F_x__root_mean_square,T_y__absolute_maximum,F_x__mean_n_absolute_max__number_of_maxima_7,F_x__range_count__max_1__min_-1,F_y__abs_energy,F_y__root_mean_square,F_y__mean_n_absolute_max__number_of_maxima_7,T_y__variance,...,"F_y__cwt_coefficients__coeff_13__w_2__widths_(2, 5, 10, 20)","F_y__cwt_coefficients__coeff_14__w_5__widths_(2, 5, 10, 20)",T_y__lempel_ziv_complexity__bins_3,T_y__quantile__q_0.1,F_z__time_reversal_asymmetry_statistic__lag_1,F_x__quantile__q_0.2,F_y__quantile__q_0.7,"T_x__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0",T_z__large_standard_deviation__r_0.35000000000000003,T_z__quantile__q_0.9
1,14.0,14.0,0.966092,1.0,1.000000,15.0,13.0,0.930949,1.000000,0.222222,...,-0.310265,-0.751682,0.400000,-1.0,-596.000000,-1.0,-1.0,0.0,0.0,0.0
2,7.0,25.0,1.290994,5.0,1.571429,13.0,76.0,2.250926,3.000000,4.222222,...,-0.202951,0.057818,0.533333,-3.6,-680.384615,-1.0,-1.0,0.0,1.0,0.0
3,11.0,12.0,0.894427,5.0,1.000000,14.0,40.0,1.632993,2.142857,3.128889,...,0.539121,0.912474,0.533333,-4.0,-617.000000,-1.0,0.0,0.0,1.0,0.0
4,5.0,16.0,1.032796,6.0,1.285714,10.0,60.0,2.000000,2.428571,7.128889,...,-2.641390,-0.609735,0.533333,-4.6,3426.307692,-1.0,1.0,0.0,0.0,0.0
5,9.0,17.0,1.064581,5.0,1.285714,13.0,46.0,1.751190,2.285714,4.160000,...,0.591927,0.072771,0.466667,-5.0,-2609.000000,-1.0,0.8,0.0,0.0,0.6


In [9]:
X_full_train, X_full_test, y_train, y_test = train_test_split(X, y, test_size=.4)
X_filtered_train, X_filtered_test = X_full_train[X_filtered.columns], X_full_test[X_filtered.columns]

In [10]:
classifier_full = DecisionTreeClassifier()
classifier_full.fit(X_full_train, y_train)
print(classification_report(y_test, classifier_full.predict(X_full_test)))

              precision    recall  f1-score   support

       False       1.00      0.97      0.98        29
        True       0.88      1.00      0.93         7

    accuracy                           0.97        36
   macro avg       0.94      0.98      0.96        36
weighted avg       0.98      0.97      0.97        36



In [11]:
classifier_filtered = DecisionTreeClassifier()
classifier_filtered.fit(X_filtered_train, y_train)
print(classification_report(y_test, classifier_filtered.predict(X_filtered_test)))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00        29
        True       1.00      1.00      1.00         7

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [11]:
import pandas as pd
df = pd.DataFrame({
"id": [1, 1, 1, 1, 1, 1],
"time": [1, 2, 3, 4, 8, 9],
"x": [1, 2, 3, 4, 10, 11],
"y": [5, 6, 7, 8, 12, 13],
})
from tsfresh.utilities.dataframe_functions import roll_time_series
df_rolled = roll_time_series(df, column_id="id",min_timeshift=2)
from tsfresh import extract_features
df_features = extract_features(df_rolled, column_id="id")

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 82.74it/s]
